## Segmenting and Clustering Neighborhoods in Toronto

#### Importing libraries

In [192]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



#### Data source

In [211]:
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipage = requests.get(link)
page = wikipage.text

In [212]:
soup = BeautifulSoup(page, 'html.parser')


column_names = ['Postcode', 'Borough', 'Neighbourhood'] 

neighbs=[]

tableSoup = soup.find_all("table")
table = tableSoup[0]  #getting the first instance table from all found tables
rows = table.find_all("tr")
for row in rows:
    cols = row.find_all("td")
    if len(cols) == 0 : continue
    neighbs.append({'Postcode': cols[0].text,'Borough': cols[1].text,'Neighbourhood': cols[2].text.split("\n")[0]})


df=pd.DataFrame(neighbs,columns=column_names)


#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 
df.columns=column_names



In [213]:

df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [214]:


df=df.drop(df[df['Borough']=='Not assigned'].index).reset_index(drop=True)


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [221]:
##replacing value conditionally
df.loc[df['Neighbourhood'] =='Not assigned', 'Neighbourhood'] = df['Borough']

In [222]:
df.shape

(103, 3)

#### Grouping by Postcode and  Borough Columns, concatenating Neighbourhoods

In [218]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [219]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [220]:
df.shape

(103, 3)